In [ ]:
# !pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'

Mounted at /content/drive


In [ ]:
import time
def current_milli_time():
    return round(time.time() * 1000)

In [ ]:
import sys
sys.path.append(rootpath+'/package')
from utils.env_sdp import env
import pickle
import numpy as np
import csv

In [ ]:
from model.uav import UAV, setUAVEnv
from model.pureSDP import calculateSDP, setSDPEnv
from model.pathlossmodel import PathLossModel, NoiseMaker, setPathLossModelEnv

from service.distributeuavs import createUAVsSamples,setDistributeUavsEnv , TestEnv
from service.measurerss import measureRSS, setMeasuerRssEnv
from service.combinationset import combinationSet , setCombinationSetEnv
from multiprocessing import Pool
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

setDistributeUavsEnv("sdp")
TestEnv()
setCombinationSetEnv("sdp")
setPathLossModelEnv("sdp")
setMeasuerRssEnv("sdp")
setUAVEnv("sdp")
setSDPEnv("sdp")

envName sdp
16


In [ ]:
savePath = '/content/drive/Shareddrives/KW phase1 repo/datas/anomalyNewSDPResult/0811_MDS'
rootPath = '/content/drive/Shareddrives/KW phase1 repo'
!mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalyNewSDPResult
!mkdir /content/drive/Shareddrives/KW\ phase1\ repo/datas/anomalyNewSDPResult/0811_MDS

mkdir: cannot create directory ‘/content/drive/Shareddrives/KW phase1 repo/datas/anomalyNewSDPResult’: File exists
mkdir: cannot create directory ‘/content/drive/Shareddrives/KW phase1 repo/datas/anomalyNewSDPResult/0811_MDS’: File exists


In [ ]:
# deviations = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
deviations = [2]
# deviations = [1,0.9,0.8,0.7,0.6]
# deviations = [0.5]

In [ ]:
GetDevicationStr = lambda deviation: str(int(deviation*100))

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
print(X.shape)
mds = MDS(2,random_state=0)
X_2d = mds.fit_transform(X)
print(X_2d.shape)

(150, 4)
(150, 2)


In [ ]:
dict_slice = lambda adict, start, end: { k:adict[k] for k in list (adict.keys())[start:end] }

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import pickle as dill
from model.uav import UAV
MAXPICKERROR = 3
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    import pickle as dill
    file = open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy","wb")
    file.close()
    file = open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy","wb")
    file.close()
    for j in range(env.N_ngps):
      file = open(savePath+"/test_position{}_pickerror{}_deviation{}_100.csv".format(j,ipickError,GetDevicationStr(deviation)),"wb")
      file.close()
      # print(ipickError)
for deviation in deviations:
  for ipickError in np.arange(0,MAXPICKERROR+1,1):
    print("\nDeviation: {}, Anomaly: {}".format(deviation, ipickError))
    _pathLossModel = PathLossModel()
    _pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = deviation) 
    with open(rootPath+"/datas/uavPosition/300_0706/deviation/"+GetDevicationStr(deviation)+"/UAVsSamples.pkl", "rb") as f:
      UAVsSamples = pickle.load(f)
    UAVsSamples = dict_slice(UAVsSamples, 0, 100)
    print(len(UAVsSamples))
    # UAVsSamples = {0:UAVsSamples[0]}
    for iSample, UAVSample in UAVsSamples.items():
      for pickTime in range(ipickError):
        # print("pickTime",pickTime)
        uavList_gps = []
        for id, uav in UAVSample.items():
          if not uav.observedPosition == None and uav.observedPosition == uav.realPosition:
            uavList_gps.append(id)
        pickedAnomalyIndex = np.random.choice(uavList_gps, 1, replace=False)[0]
        # print("pickedAnomalyIndex",pickedAnomalyIndex)
        originObservedPosition = UAVSample[pickedAnomalyIndex].observedPosition
        shift = np.random.choice(np.arange(500, 1000), 3, replace=False)
        anomalyObservedPosition = [originObservedPosition[0]+shift[0], originObservedPosition[1]+shift[1], originObservedPosition[2]+shift[2]]
        UAVSample[pickedAnomalyIndex].observedPosition = anomalyObservedPosition
    measureRSS(UAVsSamples, _pathLossModel)
    for iSample, UAVSample in UAVsSamples.items():
      # print("Sample: ", iSample)
      print(".", end="")
      uavAnchorOfSegments = []
      uavTargetOfSegments = []
      uavAnchorOfSegmentsMDS = []
      uavTargetOfSegmentsMDS = []
      groupSet = combinationSet({0:UAVSample})[0]
      # print(groupSet)

      ## for GPS data
      for anchorSize, segments in groupSet.items():
        for segment in segments:
          uavAnchors = []
          for anchorID in segment:
            uavAnchors.append(UAVSample[anchorID].observedPosition)
          for i in range(env.N_gps-len(uavAnchors)):
            uavAnchors.append([0.0,0.0,0.0])
          uavAnchorOfSegments.append(uavAnchors)
      # print(np.array(uavAnchorOfSegments).shape)
      ## for RSS data
      for anchorSize, segments in groupSet.items():
        for segment in segments:
          uavTargets = [ [] for i in range(env.N_gps)]
          iAnchor = 0
          for anchorID in segment:
            for id, uav in UAVSample.items():
              if uav.observedPosition == None:
                rss = uav.searchRSSById(anchorID)
                uavTargets[iAnchor].append(rss)
            iAnchor += 1
          for i in range(env.N_gps-iAnchor):
            # print(iAnchor)
            for id, uav in UAVSample.items():
              if uav.observedPosition == None:
                rss = uav.searchRSSById(anchorID)
                uavTargets[iAnchor].append(0)
            iAnchor += 1
          # print(uavTargets)
          # print(np.array(uavTargets).shape)
          uavTargetOfSegments.append(uavTargets)
      # print(np.array(uavTargetOfSegments).shape)
      import numpy as np
      # from sklearn.decomposition import PCA
      for anchorGPS in uavAnchorOfSegments:
        mds = MDS(n_components=1)
        # print(np.array(anchorGPS).shape)
        # print(euclidean_distances(np.array(anchorGPS), squared=True))
        # anchorGPStransfer = mds.fit_transform(euclidean_distances(np.array(anchorGPS), squared=True))
        anchorGPStransfer = mds.fit_transform(anchorGPS)
        uavAnchorOfSegmentsMDS.append(anchorGPStransfer)
      # print(np.array(uavAnchorOfSegmentsMDS).shape)
      for anchorRSS in uavTargetOfSegments:
        # print(euclidean_distances(np.array(anchorRSS), squared=True))
        mds = MDS(n_components=1)
        # anchorRSStransfer = mds.fit_transform(euclidean_distances(np.array(anchorRSS), squared=True))
        anchorRSStransfer = mds.fit_transform(anchorRSS)
        uavTargetOfSegmentsMDS.append(anchorRSStransfer)
      # print(np.array(uavTargetOfSegmentsMDS).shape)
      uavAnchorOfSegmentsMDS = np.array(uavAnchorOfSegmentsMDS)
      uavTargetOfSegmentsMDS = np.array(uavTargetOfSegmentsMDS)
      MDSdata = np.append(uavAnchorOfSegmentsMDS, uavAnchorOfSegmentsMDS,axis=2)
      MDSdata = MDSdata.transpose((1,0,2))
      # print(np.array(MDSdata).shape)
      for j in range(env.N_ngps):
        with open(savePath+"/test_position{}_pickerror{}_deviation{}_100.csv".format(j,ipickError,GetDevicationStr(deviation)), "a") as _csv:
          csvWriter = csv.writer(_csv, delimiter=',')
          csvWriter.writerows(MDSdata[j])
      anomalyUAVs = []
      for iUAV, UAV in UAVSample.items():
        if UAV.observedPosition != None and UAV.observedPosition != UAV.realPosition:
          # print(UAV.observedPosition)
          # print(UAV.realPosition)
          anomalyUAVs.append(UAV.id)
      with open(savePath+"/anomalyUAVs_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy", 'ab') as handle:
        dill.dump(anomalyUAVs, handle)
      with open(savePath+"/UAVsSamples_pick"+str(ipickError)+"div"+GetDevicationStr(deviation)+"_100.npy", 'ab') as handle:
        dill.dump(UAVSample, handle)


Deviation: 2, Anomaly: 0
100
....................................................................................................
Deviation: 2, Anomaly: 1
100
....................................................................................................
Deviation: 2, Anomaly: 2
100
....................................................................................................
Deviation: 2, Anomaly: 3
100
....................................................................................................